In [31]:
import pandas as pd
import csv
from utilities import map_aspect_values_to_movies,dict_movie_aspect
from operator import itemgetter
def change_author(x):
    qunima = itemgetter(*x)(authors_dict)
    if type(qunima) == int:
        return [qunima]
    else:
        return change_list(qunima)
    
def change_genre(x):
    qunima = itemgetter(*x)(genres_dict)
    if type(qunima) == int:
        return [qunima]
    else:
        return change_list(qunima)    

def change_list(x):
    return list(x)

In [17]:
train = pd.read_csv('train.dat',sep='\t',header=None)
train = train.rename(columns = {0:'user',1:'item',2:'rating'})
test = pd.read_csv('test.dat',sep='\t',header=None)
test = test.rename(columns = {0:'user',1:'item',2:'rating'})
validation = pd.read_csv('valid.dat',sep='\t',header=None)
validation = validation.rename(columns = {0:'user',1:'item',2:'rating'})
df = pd.concat([train,test,validation])

In [18]:
jiba = pd.read_csv('i_map.dat',sep = '\t',header = None)
jiba

haha = pd.read_csv('i2kg_map.tsv',sep='\t', header=None)
haha = haha.rename(columns = {0:'item',1:'name',2:'url'})
haha = haha.sort_values(by='item',ascending=True)

In [19]:
haha = pd.read_csv('i2kg_map.tsv',sep='\t', header=None)
haha = haha.rename(columns = {0:'item',1:'name',2:'url'})
haha = haha.sort_values(by='item',ascending=True)

In [20]:
haha_final = haha[haha['item'].isin(jiba[1])]

In [21]:
with open("attribute.json", 'r', encoding='UTF-8') as f:
    info = json.load(f)
info = {int(k): v for k, v in info.items()}

In [22]:
book_genres = dict_movie_aspect(info, "genre")
book_authors = dict_movie_aspect(info,'author')
authors = pd.DataFrame.from_dict(book_authors,dtype = 'int64',orient = 'index').T.replace(np.nan, 0).index
genres = pd.DataFrame.from_dict(book_genres,dtype = 'int64',orient = 'index').T.replace(np.nan, 0).index

<ipython-input-22-23b6162982d9>:3: FutureWarning: In a future version, passing float-dtype values containing NaN and an integer dtype will raise IntCastingNaNError (subclass of ValueError) instead of silently ignoring the passed dtype. To retain the old behavior, call Series(arr) or DataFrame(arr) without passing a dtype.
  authors = pd.DataFrame.from_dict(book_authors,dtype = 'int64',orient = 'index').T.replace(np.nan, 0).index
<ipython-input-22-23b6162982d9>:4: FutureWarning: In a future version, passing float-dtype values containing NaN and an integer dtype will raise IntCastingNaNError (subclass of ValueError) instead of silently ignoring the passed dtype. To retain the old behavior, call Series(arr) or DataFrame(arr) without passing a dtype.
  genres = pd.DataFrame.from_dict(book_genres,dtype = 'int64',orient = 'index').T.replace(np.nan, 0).index


In [23]:
haha_final['authors'] = haha_final['item'].map(info)
haha_final['genres'] = haha_final['item'].map(info)

<ipython-input-23-4dc8e185da26>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  haha_final['authors'] = haha_final['item'].map(info)
<ipython-input-23-4dc8e185da26>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  haha_final['genres'] = haha_final['item'].map(info)


In [24]:
for i in haha_final.index:
    haha_final['authors'][i] = info[haha_final['item'][i]]['author']
    haha_final['genres'][i] = info[haha_final['item'][i]]['genre']

<ipython-input-24-c231ed7d4dfb>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  haha_final['authors'][i] = info[haha_final['item'][i]]['author']
<ipython-input-24-c231ed7d4dfb>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  haha_final['genres'][i] = info[haha_final['item'][i]]['genre']


In [25]:
haha_final.index = range(len(haha_final))
#del haha_final['url']
jiba = jiba.rename(columns = {0:'real',1:'item'})
final = pd.merge(haha_final,jiba)

In [26]:
del final['item']
final = final.rename(columns = {"real":"item"})
model_df = pd.merge(df,final)
model_df = model_df[['user', 'item', 'authors', 'genres','rating']]
authors_dict = dict(zip(authors, range(len(authors))))
genres_dict = dict(zip(genres, range(
    len(authors), len(authors) + len(genres))))

In [32]:
hehe = model_df.copy()
hehe['authors'] = hehe['authors'].map(change_author)
hehe['genres'] = hehe['genres'].map(change_genre)

In [ ]:
hehe_test = hehe.copy()
df_empty = pd.DataFrame(columns=['user', 'item', 'authors', 'genres', 'rating'])
for user in set(hehe_test['user']):
    mean_rating = hehe_test[hehe_test['user'] == user]['rating'].mean()
    pos_df = hehe_test[hehe_test['user'] == user]
    pos_df = pos_df[pos_df['rating'] > mean_rating]
    pos_df = pos_df.iloc[-1:]
    index_to_remove = list(pos_df.iloc[-1:].index)
    df_empty = df_empty.append(pos_df)
    hehe_test.drop(index = index_to_remove)
    
    